In [1]:
import time
import numpy as np
from ibllib.atlas.regions import BrainRegions
from one.api import ONE
import ibllib.atlas as atlas
import pickle

# have to do this because the Examples folder is below the unitymouse folder
import sys
sys.path.append("..") 
import unitymouse.render as umr
"""
Display all trajectories in one single brain volume
Author: Mayo, Gaelle, Dan
"""

import numpy as np

one = ONE()
ba = atlas.AllenAtlas(25)


C:\Users\Dan\anaconda3\envs\iblenv\lib\site-packages\one\api.py:1242: UserWarning: Newer cache tables require ONE version 1.10.0 or greater
  warnings.warn(f'Newer cache tables require ONE version {min_version} or greater')


In [2]:
umr.setup(localhost = True, standalone = True)

UnityMouse Renderer connected to server
Login sent with ID: Dan


In [41]:
trajectories = one.alyx.rest('trajectories', 'list', provenance='Planned', x=-2243, y=-2000, theta=15,
                                   project='ibl_neuropixel_brainwide_01', use_cache=False)

In [37]:
def getCoords(ins):
    entry_coords = ba.xyz2ccf(ins.entry)
    tip_coords = ba.xyz2ccf(ins.tip)
    tip_angles = [ins.phi, ins.theta, ins.beta]
    recording_size = np.sqrt(np.sum(np.power(entry_coords-tip_coords,2)))
    return (tip_coords, tip_angles, recording_size)

In [42]:
#This block is only needed if you are re-running this from scratch, otherwise just load the .pkl file

# Pull the micro-manip and histology tracks for each of these recordings
data = {}



for traj in trajectories:
    pid = traj['probe_insertion']
    
    
    traj_histology = one.alyx.rest('trajectories', 'list',
        provenance='Ephys aligned histology track',
        probe_insertion=pid, use_cache=False)

    if len(traj_histology) == 0:
        traj_histology = one.alyx.rest('trajectories', 'list', provenance='Histology track',
            probe_insertion=pid, use_cache=False)
        
    traj_micro = one.alyx.rest('trajectories', 'list',
        provenance='Micro-manipulator',
        probe_insertion=pid, use_cahce=False)

    # skip any sessions that we are missing histology for
    if len(traj_histology) == 0:
        continue

    if not traj_histology[0]['x']:
        continue

    ins_planned = atlas.Insertion.from_dict(traj)
    ins_micro = atlas.Insertion.from_dict(traj_histology[0])
    ins_histology = atlas.Insertion.from_dict(traj_micro[0])
    
    

    data[pid] = {}
    data[pid]['planned'] = getCoords(ins_planned)
    data[pid]['micro-manip'] = getCoords(ins_micro)
    data[pid]['histology'] = getCoords(ins_histology)
    
pickle.dump(data,open( './data/gaelle/ibl_repro_ephys_traj.pkl', "wb" ))

In [3]:
data = pickle.load( open( "./data/gaelle/ibl_repro_ephys_traj.pkl", "rb" ) )


In [4]:
import random
r = lambda: random.randint(0,255)
randHexColor = lambda : '#%02X%02X%02X' % (r(),r(),r())

In [5]:
# VIEW 1: Probe positions planned vs micro-manipulator
umr.clear()
count = 0

vdata = {'VISa': True, 'CA1': True, 'DG': True, 'LP': True, 'PO': True}
umr.set_area_visibility(vdata)
vdata = {'VISa': 'default', 'CA1': 'default', 'DG': 'default', 'LP': 'default', 'PO': 'default'}
umr.set_area_shader(vdata)
vdata = {'VISa': 0.4, 'CA1': 0.4, 'DG': 0.4, 'LP': 0.4, 'PO': 0.4}
umr.set_area_alpha(vdata)

for i, (pid, trajs) in enumerate(data.items()):
    print(pid)
    for j, (traj_type, traj) in enumerate(trajs.items()):
        if traj_type == "planned":
            probename = 'p'+str(count)
            
            tip_coords = traj[0]
            tip_angles = traj[1]
            recording_size = traj[2]

            count+=1
            umr.create_probes([probename])
            umr.set_probe_positions({probename:tip_coords.tolist()})
            umr.set_probe_colors({probename:'#F28C28'})
            umr.set_probe_angles({probename:tip_angles})
            umr.set_probe_size({probename:[0.07,recording_size/1000,0.02]})


AttributeError: module 'unitymouse.render' has no attribute 'set_area_visibility'

In [23]:
umr.clear()
umr.set_allen_volume_visibility(True)

# temp_traj_rep = [traj_rep[0]]
count = 0

for i, (pid, traj) in enumerate(data.items()):
    tip_coords = traj[0]
    tip_angles = traj[1]
    recording_size = traj[2]
    
    probename = 'p'+str(count)
    count+=1
    umr.create_probes([probename])
    umr.set_probe_positions({probename:tip_coords.tolist()})
    umr.set_probe_colors({probename:'#F28C28'})
    umr.set_probe_angles({probename:tip_angles})
    umr.set_probe_size({probename:[0.07,recording_size/1000,0.02]})

vdata = {'VISa': True, 'CA1': True, 'DG': True, 'LP': True, 'PO': True}
umr.set_volume_visibility(vdata)
vdata = {'VISa': 'default', 'CA1': 'default', 'DG': 'default', 'LP': 'default', 'PO': 'default'}
umr.set_volume_shader(vdata)
vdata = {'VISa': 0.4, 'CA1': 0.4, 'DG': 0.4, 'LP': 0.4, 'PO': 0.4}
umr.set_volume_alpha(vdata)

# # for some reason 'root' doesn't work?
# umr.set_volume_visibility({8:True})
# umr.set_volume_shader({8:'transparent-unlit'})
# umr.set_volume_color({8:'#000000'})
# umr.set_volume_alpha({8:0.025})

In [15]:
umr.close()

UnityMouse Renderer connected to server
Login sent with ID: Dan
